In [ ]:
import sys  


In [ ]:
!{sys.executable} -m pip install --user py2neo


In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%matplotlib inline

In [6]:
df = pd.read_csv("data/preprocessed.csv")
df.head()

,customer,merchant,category,amount,fraud
0,'C1093826151','M348934600','es_transportation',4.55,0
1,'C352968107','M348934600','es_transportation',39.68,0
2,'C2054744914','M1823072687','es_transportation',26.89,0
3,'C1760612790','M348934600','es_transportation',17.25,0
4,'C757503768','M348934600','es_transportation',35.72,0


In [9]:
#Feature scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['amount', 'fraud']] = scaler.fit_transform(df[['amount', 'fraud']])

In [10]:
# split the features and labels
label = df.fraud
features = df.drop('fraud', axis = 1)
features.head()

,customer,merchant,category,amount
0,'C1093826151','M348934600','es_transportation',0.000546
1,'C352968107','M348934600','es_transportation',0.004764
2,'C2054744914','M1823072687','es_transportation',0.003228
3,'C1760612790','M348934600','es_transportation',0.002071
4,'C757503768','M348934600','es_transportation',0.004288


In [13]:
#One hot encoding
features = pd.get_dummies(features)

In [15]:
#Splitting data into training data and testing data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, label, train_size = 0.8, random_state = 0)

In [16]:
from py2neo import Graph

graph = Graph(password="semanticweb", bolt_port=7687, http_port=7474)

In [17]:
# Use cypher query to get data from all the nodes 
query = """
MATCH (p:Placeholder)
RETURN p.id AS id, p.degree AS degree, p.pagerank as pagerank, p.community AS community 
"""

data = graph.run(query)
valueDict = {}
for d in data:
    valueDict[d['id']] = {'degree': d['degree'], 'pagerank': d['pagerank'], 'community': d['community']}
    
print(valueDict)

{"'M348934600'": {'degree': 11787, 'pagerank': 139.32018184661865, 'community': 608498}, "'M1823072687'": {'degree': 10719, 'pagerank': 123.8707223892212, 'community': 608499}, "'M50039827'": {'degree': 1800, 'pagerank': 19.511408519744872, 'community': 608500}, "'M1888755466'": {'degree': 2004, 'pagerank': 20.411254549026488, 'community': 608501}, "'M1053599405'": {'degree': 8487, 'pagerank': 97.01495351791382, 'community': 608502}, "'M85975013'": {'degree': 10326, 'pagerank': 119.2847936630249, 'community': 608503}, "'M480139044'": {'degree': 3972, 'pagerank': 41.82132668495178, 'community': 608504}, "'M692898500'": {'degree': 1647, 'pagerank': 18.081495594978332, 'community': 608505}, "'M348875670'": {'degree': 297, 'pagerank': 3.2908756911754606, 'community': 608506}, "'M151143676'": {'degree': 8184, 'pagerank': 94.09342465400697, 'community': 608507}, "'M840466850'": {'degree': 3393, 'pagerank': 36.986967897415155, 'community': 608508}, "'M1741626453'": {'degree': 1296, 'pagerank'

In [26]:
# Helper functions to add network features to input dataframe 
def add_degree(x):
    return valueDict[x]['degree']
def add_community(x):
    return str(valueDict[x]['community']) # cast to string for one-hot encoding
def add_pagerank(x):
    return valueDict[x]['pagerank']

In [27]:
df = pd.read_csv("data/preprocessed.csv")
df['merchDegree'] = df.merchant.apply(add_degree)
df['custDegree'] = df.customer.apply(add_degree)
df['custPageRank'] = df.customer.apply(add_pagerank)
df['merchPageRank'] = df.merchant.apply(add_pagerank)
df['merchCommunity'] = df.merchant.apply(add_community)
df['custCommunity'] = df.customer.apply(add_community)

In [28]:
df.head()

,customer,merchant,category,amount,fraud,merchDegree,custDegree,custPageRank,merchPageRank,merchCommunity,custCommunity
0,'C1093826151','M348934600','es_transportation',4.55,0,11787,7,0.15,139.320182,608498,608498
1,'C352968107','M348934600','es_transportation',39.68,0,11787,12,0.15,139.320182,608498,608498
2,'C2054744914','M1823072687','es_transportation',26.89,0,10719,14,0.15,123.870722,608499,608498
3,'C1760612790','M348934600','es_transportation',17.25,0,11787,11,0.15,139.320182,608498,608498
4,'C757503768','M348934600','es_transportation',35.72,0,11787,14,0.15,139.320182,608498,608498


In [29]:
features_graph = df.drop('fraud', axis = 1)


In [30]:
features_graph[['amount', 'merchDegree', 'custDegree', 'custPageRank', 'merchPageRank']].head()


,amount,merchDegree,custDegree,custPageRank,merchPageRank
0,4.55,11787,7,0.15,139.320182
1,39.68,11787,12,0.15,139.320182
2,26.89,10719,14,0.15,123.870722
3,17.25,11787,11,0.15,139.320182
4,35.72,11787,14,0.15,139.320182


In [31]:
scaler = MinMaxScaler()

df[['amount', 'merchDegree', 'custDegree', 'custPageRank', 'merchPageRank']] = scaler.fit_transform(df[['amount', 'merchDegree', 'custDegree', 'custPageRank', 'merchPageRank']])
features_graph[['amount', 'merchDegree', 'custDegree', 'custPageRank', 'merchPageRank']] = df[['amount', 'merchDegree', 'custDegree', 'custPageRank', 'merchPageRank']]


# Show an example of a record with scaling applied
features_graph.head()

,customer,merchant,category,amount,merchDegree,custDegree,custPageRank,merchPageRank,merchCommunity,custCommunity
0,'C1093826151','M348934600','es_transportation',0.000546,1.000000,0.172414,0.0,1.000000,608498,608498
1,'C352968107','M348934600','es_transportation',0.004764,1.000000,0.344828,0.0,1.000000,608498,608498
2,'C2054744914','M1823072687','es_transportation',0.003228,0.909322,0.413793,0.0,0.888913,608499,608498
3,'C1760612790','M348934600','es_transportation',0.002071,1.000000,0.310345,0.0,1.000000,608498,608498
4,'C757503768','M348934600','es_transportation',0.004288,1.000000,0.413793,0.0,1.000000,608498,608498


In [32]:
features_enhanced = pd.get_dummies(features_graph)


In [33]:
features_enhanced.head()

,amount,merchDegree,custDegree,custPageRank,merchPageRank,customer_'C1000148617',customer_'C100045114',customer_'C1000699316',customer_'C1001065306',customer_'C1002658784',...,merchCommunity_608544,merchCommunity_608545,merchCommunity_608546,merchCommunity_608547,custCommunity_608498,custCommunity_608499,custCommunity_608500,custCommunity_608501,custCommunity_608504,custCommunity_608505
0,0.000546,1.000000,0.172414,0.0,1.000000,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0.004764,1.000000,0.344828,0.0,1.000000,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0.003228,0.909322,0.413793,0.0,0.888913,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0.002071,1.000000,0.310345,0.0,1.000000,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.004288,1.000000,0.413793,0.0,1.000000,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [34]:
features_enhanced.shape


(594643, 4238)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, auc
from sklearn import metrics
from sklearn.model_selection import train_test_split

X_train_enh, X_test_enh, y_train_enh, y_test_enh = train_test_split(features_enhanced, 
                                                    label, 
                                                    test_size = 0.2, 
                                                    random_state = 5)

rfc = RandomForestClassifier(n_estimators = 15, max_depth = 19, criterion = "entropy", random_state = 0, min_samples_split = 20)
rfc.fit(X_train_enh, y_train_enh)
#Predict for x_test
y_pred = rfc.predict(X_test_enh)
#Checking accuracy of Naive Bayes
cm = metrics.confusion_matrix(y_test_enh, y_pred) 
print(cm)
acc = metrics.accuracy_score(y_test_enh, y_pred) 
print("Accuracy score:",acc)
#Creating a classification report
cr = classification_report(y_test_enh, y_pred)
print(cr)